**Preguntas que buscamos responder**

Análisis de calidad de datos:
1. ¿Qué IDs de sensor o Silos/Ubicaciones presentan el mayor número de valores faltantes (NULLs/vacíos)
2. ¿Existen mediciones de Temperatura o CO2 que sean anómalas (muy altas o muy bajas) y no estén asociadas a niveles altos de humedad?

Requerimiento del usuario:

> Registrar y notificar automáticamente al usuario cuando se detecten condiciones anómalas en las variables H, T y CO2, con el objeto de accionar preventivamente, evitando o minimizando daños en los granos almacenados

Análisis de cumplimiento y localización de riesgo:
1. ¿Qué porcentaje de las mediciones de humedad están en estado 'Normal' (HR 63%), 'Preventivo' (HR entre 63% y 67%), y 'Alarma' (HR > 67%)?
2. ¿Cuál es el Silo (1, 2 o 3) y la Ubicación (Superior, Medio, Inferior) que registra la mayor frecuencia o persistencia de lecturas en rango 'Alarma' (HR > 67%)?
3. ¿En qué fechas/horas se detectó un gradiente de humedad (diferencia) superior a 5% HR entre dos puntos de medición del mismo silo (por ejemplo, Superior vs. Inferior)?
4. ¿Este gradiente se asocia a un patrón de riesgo específico?

Detección de Eventos y Tendencias Críticas (Temporal)

Estas preguntas buscan identificar los focos de deterioro y los problemas de los sensores.

1. Identificar todos los casos donde la HR en un sensor específico haya mostrado un incremento sostenido (por ejemplo, un aumento mayor al 2% en un periodo de 6 horas). ¿Estos incrementos ocurren con mayor frecuencia en ciertas horas del día (patrones diarios)?  
2. ¿Se observa una correlación negativa significativa donde la Temperatura disminuye y la Humedad Relativa aumenta?
3. ¿Existe una correlación positiva entre la HR alta (HR > 63%) y los niveles de CO2 en los mismos puntos de medición?









In [1]:
# Importamos las librerías necesarias para el desarrollo del proyecto
import pandas as pd

#⚙️ **PROCESO ETL**

Llevamos adelante el proceso de extracción, transformación y carga de los datos contenidos en el dataset original. Este proceso tuvo como objetivo recuperar y preparar la información para su análisis posterior, asegurando la integridad y confiabilidad de los datos.

🗂️ **EXTRACT**

En esta etapa se extrajeron los datos desde la fuente original y se cargaron en un DataFrame de pandas, que servirá como estructura de datos central para el análisis posterior.
Cada fila representa un registro de medición y cada columna detalla las características de ese registro, lo que permite explorar la información de manera ordenada y eficiente.

Las acciones realizadas en esta etapa fueron:

*   **Carga de datos**: se importó el dataset y se almacenaron los registros en un DataFrame.
*   **Inspección inicial y revisión de la estructura**: se analizaron los primeros registros, la información general de dataset, la cantidad de filas/columnas, y los tipos de dato para asegurar compatibilidad con la etapa de análisis.
*   **Revisión de la integridad de los datos**: se identificaron valores faltantes, se revisaron valores únicos en columnas clave para conocer los componentes del sistema IoT, y se detectaron registros duplicados exactos.

Este proceso no solo fue fundamental para acceder a los datos con los que se trabajará, sino que también permitió definir las acciones a realizar en la etapa de Transform.



In [3]:
# Leemos los datos del dataset y los almacenamos en un DataFrame
# No se especifica 'index_col' para generar un índice autoincremental
#montamos el drive

df = pd.read_csv('dataset_sensores.csv')

In [4]:
# Leemos los primeros 5 registros
df.head(5)

,Fecha y hora,ID sensor,ID silo,Ubicación,Temperatura (°C),Humedad (%),CO2 (%)
0,2025-06-01 00:00:00,1,Silo 1,superior,13.494586,8.189056,3.164652
1,2025-06-01 00:00:00,2,Silo 1,medio,11.210830,8.508008,2.728765
2,2025-06-01 00:00:00,3,Silo 1,inferior,14.311821,8.993510,2.550330
3,2025-06-01 00:00:00,4,Silo 2,superior,14.289996,10.565392,3.738029
4,2025-06-01 00:00:00,5,Silo 2,medio,14.420994,9.908821,3.247908


In [5]:
# Visualizamos la información general del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2764 entries, 0 to 2763
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fecha y hora      2764 non-null   object 
 1   ID sensor         2764 non-null   int64  
 2   ID silo           2764 non-null   object 
 3   Ubicación         2764 non-null   object 
 4   Temperatura (°C)  2522 non-null   float64
 5   Humedad (%)       2533 non-null   float64
 6   CO2 (%)           2535 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 151.3+ KB


In [6]:
# Averiguamos el total de filas y columnas
df.shape

(2764, 7)

In [7]:
# Inspeccionamos el tipo de dato de las columnas
df.dtypes

Fecha y hora         object
ID sensor             int64
ID silo              object
Ubicación            object
Temperatura (°C)    float64
Humedad (%)         float64
CO2 (%)             float64
dtype: object

In [8]:
# Detección de valores faltantes (NaN) por columna
df.isnull().sum()

Fecha y hora          0
ID sensor             0
ID silo               0
Ubicación             0
Temperatura (°C)    242
Humedad (%)         231
CO2 (%)             229
dtype: int64

In [9]:
# Detección de valores únicos en columnas clave: 'Fecha y hora', 'ID sensor', 'ID silo' y 'Ubicación'.
df["Fecha y hora"].unique()

array(['2025-06-01 00:00:00', '2025-06-01 12:00:00',
       '2025-06-02 00:00:00', '2025-06-02 12:00:00',
       '2025-06-03 00:00:00', '2025-06-03 12:00:00',
       '2025-06-04 00:00:00', '2025-06-04 12:00:00',
       '2025-06-05 00:00:00', '2025-06-05 12:00:00',
       '2025-06-06 00:00:00', '2025-06-06 12:00:00',
       '2025-06-07 00:00:00', '2025-06-07 12:00:00',
       '2025-06-08 00:00:00', '2025-06-08 12:00:00',
       '2025-06-09 00:00:00', '2025-06-09 12:00:00',
       '2025-06-10 00:00:00', '2025-06-10 12:00:00',
       '2025-06-11 00:00:00', '2025-06-11 12:00:00',
       '2025-06-12 00:00:00', '2025-06-12 12:00:00',
       '2025-06-13 00:00:00', '2025-06-13 12:00:00',
       '2025-06-14 00:00:00', '2025-06-14 12:00:00',
       '2025-06-15 00:00:00', '2025-06-15 12:00:00',
       '2025-06-16 00:00:00', '2025-06-16 12:00:00',
       '2025-06-17 00:00:00', '2025-06-17 12:00:00',
       '2025-06-18 00:00:00', '2025-06-18 12:00:00',
       '2025-06-19 00:00:00', '2025-06-19 12:0

In [10]:
df["ID sensor"].unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [11]:
df["ID silo"].unique()

array(['Silo 1', 'Silo 2', 'Silo 3'], dtype=object)

In [12]:
df["Ubicación"].unique()

array(['superior', 'medio', 'inferior'], dtype=object)

In [13]:
# Identificación de registros duplicados exactos
df[df.duplicated(keep=False)]

,Fecha y hora,ID sensor,ID silo,Ubicación,Temperatura (°C),Humedad (%),CO2 (%)
0,2025-06-01 00:00:00,1,Silo 1,superior,13.494586,8.189056,3.164652
1,2025-06-01 00:00:00,2,Silo 1,medio,11.210830,8.508008,2.728765
2,2025-06-01 00:00:00,3,Silo 1,inferior,14.311821,8.993510,2.550330
3,2025-06-01 00:00:00,4,Silo 2,superior,14.289996,10.565392,3.738029
4,2025-06-01 00:00:00,5,Silo 2,medio,14.420994,9.908821,3.247908
5,2025-06-01 00:00:00,6,Silo 2,inferior,13.969569,8.353516,3.983539
6,2025-06-01 00:00:00,7,Silo 3,superior,14.928864,9.829876,2.511818
7,2025-06-01 00:00:00,8,Silo 3,medio,13.820210,11.210931,3.916386
8,2025-06-01 00:00:00,9,Silo 3,inferior,11.495531,9.540807,2.519978
9,2025-06-01 12:00:00,1,Silo 1,superior,10.789627,9.241363,3.890861


In [14]:
# Agrupamiento de filas idénticas y conteo de cuántas repeticiones exactas tiene cada grupo
df.groupby(list(df.columns)).size().loc[lambda x: x > 1]

Fecha y hora         ID sensor  ID silo  Ubicación  Temperatura (°C)  Humedad (%)  CO2 (%) 
2025-06-01 00:00:00  1          Silo 1   superior   13.494586         8.189056     3.164652    2
                     2          Silo 1   medio      11.210830         8.508008     2.728765    2
                     3          Silo 1   inferior   14.311821         8.993510     2.550330    2
                     4          Silo 2   superior   14.289996         10.565392    3.738029    2
                     5          Silo 2   medio      14.420994         9.908821     3.247908    2
                     6          Silo 2   inferior   13.969569         8.353516     3.983539    2
                     7          Silo 3   superior   14.928864         9.829876     2.511818    2
                     8          Silo 3   medio      13.820210         11.210931    3.916386    2
                     9          Silo 3   inferior   11.495531         9.540807     2.519978    2
2025-06-01 12:00:00  1          Sil

🧰 **TRANSFORM**

Operaciones posibles:

*   Cambiar los nombres de las columnas para evitar espacios o tildes. (df.rename)
*   Conversión de tipo de dato: convertir  columna 'Fecha y hora' a tipo datetime (pd.to_datetime). Evaluar si las columnas de temperatura, humedad y CO2 requerirían algún tipo de conversión, porque los valores que contienen son demasiado grandes, tendríamos que ver cómo llevar esos valores a un rango de 0 a 100, por ejemplo.
*   Definir qué tratamiento haremos de los valores nulos: imputarlos usando df.fillna, llenando la celda vacía con otro valor - la media, la mediana, o 'desconocido' -; eliminarlos usando df.dropna, etc.)
*   Eliminar registros idénticos duplicados [df.drop_duplicates()]





In [16]:
#Comprobaciones antes de transformar (confirmar que el archivo se cargó y ver tipos iniciales y valores de muestra.)

import pandas as pd

# Leer el archivo
df = pd.read_csv('dataset_sensores.csv')

# Chequeos rápidos
print("Shape (filas, columnas):", df.shape)
display(df.head(5))
df.info()


Shape (filas, columnas): (2764, 7)


,Fecha y hora,ID sensor,ID silo,Ubicación,Temperatura (°C),Humedad (%),CO2 (%)
0,2025-06-01 00:00:00,1,Silo 1,superior,13.494586,8.189056,3.164652
1,2025-06-01 00:00:00,2,Silo 1,medio,11.210830,8.508008,2.728765
2,2025-06-01 00:00:00,3,Silo 1,inferior,14.311821,8.993510,2.550330
3,2025-06-01 00:00:00,4,Silo 2,superior,14.289996,10.565392,3.738029
4,2025-06-01 00:00:00,5,Silo 2,medio,14.420994,9.908821,3.247908


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2764 entries, 0 to 2763
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fecha y hora      2764 non-null   object 
 1   ID sensor         2764 non-null   int64  
 2   ID silo           2764 non-null   object 
 3   Ubicación         2764 non-null   object 
 4   Temperatura (°C)  2522 non-null   float64
 5   Humedad (%)       2533 non-null   float64
 6   CO2 (%)           2535 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 151.3+ KB


In [19]:

# Normalizar todos los nombres (para evitar problemas como: nombres con espacios, mayúsculas, tildes o caracteres raros dificultan el acceso por df.col.)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Verificamos
print(df.columns.tolist())

['fecha_y_hora', 'id_sensor', 'id_silo', 'ubicacion', 'temperatura_(c)', 'humedad_(%)', 'co2_(%)']


In [25]:
#Convertir la columna de fecha/hora a tipo datetime (para poder ordenar por tiempo, agrupar por día/hora, o graficar series temporales)

df['Fecha y hora'] = pd.to_datetime(df['Fecha y hora'], errors='coerce', dayfirst=True)

# Verificá cuántos se convirtieron mal (NaT)
print("Fechas inválidas:", df['Fecha y hora'].isna().sum())

Fechas inválidas: 1674


In [26]:
# Verificación
df['Fecha y hora'].head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2764 entries, 0 to 2763
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Fecha y hora      1090 non-null   datetime64[ns]
 1   ID sensor         2764 non-null   int64         
 2   ID silo           2764 non-null   object        
 3   Ubicación         2764 non-null   object        
 4   Temperatura (°C)  2522 non-null   float64       
 5   Humedad (%)       2533 non-null   float64       
 6   CO2 (%)           2535 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 151.3+ KB


In [31]:
#Convertir columnas numéricas (temperatura, humedad, CO2) a tipo numérico para poder calcular luego medias, medianas, y escalar valores

cols_num = ['Temperatura (°C)', 'Humedad (%)', 'CO2 (%)']
for c in cols_num:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Chequeo de nulos resultantes
print(df[cols_num].isnull().sum())
df[cols_num].describe()


Temperatura (°C)    242
Humedad (%)         231
CO2 (%)             229
dtype: int64


,Temperatura (°C),Humedad (%),CO2 (%)
count,2522.000000,2533.000000,2535.000000
mean,12.740412,10.203595,3.254031
std,2.300642,1.806859,0.596173
min,5.092145,5.100860,1.005183
25%,11.225412,8.975195,2.824550
50%,12.560414,10.040795,3.249065
75%,13.931851,11.157700,3.669371
max,24.859438,19.816532,5.996912


In [35]:
# Escalar o normalizar (opcional) a rango 0-100
# no es común normalizar CO2 a 0-100 salvo que sepas el rango esperado (a discutir con el grupo)

def min_max_0_100(series):
    minv = series.min(skipna=True)
    maxv = series.max(skipna=True)
    return (series - minv) / (maxv - minv) * 100

# Ejemplo: crear columnas normalizadas solo si tiene sentido
df['Temperatura (°C)'] = min_max_0_100(df['Temperatura (°C)'])
df['Humedad (%)'] = min_max_0_100(df['Humedad (%)'])


In [39]:
# Verificación
df[['Temperatura (°C)','Humedad (%)']].head(10)

,Temperatura (°C),Humedad (%)
0,42.506784,20.985764
1,30.953579,23.153194
2,46.641063,26.452409
3,46.530653,37.134095
4,47.193356,32.672382
5,44.909658,22.103349
6,49.762601,32.135919
7,44.154071,41.520840
8,32.393842,30.171551
9,28.822771,28.136688


Tratamiento de valores faltantes / duplicados

In [40]:
# Numéricos: usar la MEDIANA (resiste valores extremos)

for c in cols_num:
    if c in df.columns:
        med = df[c].median(skipna=True)
        df[c] = df[c].fillna(med)


In [41]:
# Categóricos: reemplazar nulos con 'desconocido'

cols_cat = ['id_silo', 'ubicacion', 'id_sensor']
for c in cols_cat:
    if c in df.columns:
        df[c] = df[c].fillna('desconocido')


In [42]:
# Fechas: eliminar filas con fecha no válida

if 'fecha_hora' in df.columns:
    df = df.dropna(subset=['fecha_hora'])


In [43]:
# Eliminar registros duplicados (todas las columnas iguales)
df = df.drop_duplicates()


Ordenamos

In [44]:
# Ordenar por fecha y reiniciar el índice

if 'fecha_hora' in df.columns:
    df = df.sort_values('fecha_hora').reset_index(drop=True)


In [45]:
# Comprobación final

print("\nResumen después de limpiar:")
print(df.info())
print("\nNulos por columna:")
print(df.isnull().sum())
display(df.head())



Resumen después de limpiar:
<class 'pandas.core.frame.DataFrame'>
Index: 2720 entries, 0 to 2753
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Fecha y hora      1080 non-null   datetime64[ns]
 1   ID sensor         2720 non-null   int64         
 2   ID silo           2720 non-null   object        
 3   Ubicación         2720 non-null   object        
 4   Temperatura (°C)  2720 non-null   float64       
 5   Humedad (%)       2720 non-null   float64       
 6   CO2 (%)           2720 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 170.0+ KB
None

Nulos por columna:
Fecha y hora        1640
ID sensor              0
ID silo                0
Ubicación              0
Temperatura (°C)       0
Humedad (%)            0
CO2 (%)                0
dtype: int64


,Fecha y hora,ID sensor,ID silo,Ubicación,Temperatura (°C),Humedad (%),CO2 (%)
0,2025-01-06,1,Silo 1,superior,42.506784,20.985764,3.164652
1,2025-01-06,2,Silo 1,medio,30.953579,23.153194,2.728765
2,2025-01-06,3,Silo 1,inferior,46.641063,26.452409,2.550330
3,2025-01-06,4,Silo 2,superior,46.530653,37.134095,3.738029
4,2025-01-06,5,Silo 2,medio,47.193356,32.672382,3.247908
